In [1]:
# Here are the libraries I am using
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from os import path
import json
import csv
import networkx as nx

## Network construction

In [2]:
df_cleaned = pd.read_csv('cleaned_2015_SP_candidates.csv')

In [3]:
df_cleaned.head()

,Unnamed: 0,created_at,text,screen_name,followers,friends,rt,fav,mentioned,quoted,reweet
0,0,Thu Mar 19 21:19:03 +0000 2015,b'@selosul meet up soon for a catch up?',Helenceri,147,295,0,0,Selene O'Sullivan,NaN,NaN
1,1,Tue Sep 23 14:20:34 +0000 2014,"b""If I was Ed Miliband I wouldn't mention the ...",Helenceri,147,295,1,0,NaN,NaN,NaN
2,2,Thu Sep 11 22:20:13 +0000 2014,"b""@glenniemalcolm that's a crap argument.""",Helenceri,147,295,0,0,Malcolm Glennie,NaN,NaN
3,3,Fri Jun 27 22:44:04 +0000 2014,b'Jesus this #glastonbury coverage is so borin...,Helenceri,147,295,0,1,NaN,NaN,NaN
4,4,Fri Jun 20 22:22:43 +0000 2014,"b""Can someone tell me why Chrissie hynde has a...",Helenceri,147,295,0,0,NaN,NaN,NaN


In [4]:
len(df_cleaned)

780805

In [7]:
index = []
for i in range(len(df_cleaned['mentioned'])):
    if  pd.isnull(df_cleaned['mentioned'][i]) is False:
        index.append(i)

In [8]:
df_mention = df_cleaned.iloc[index,:].reset_index(drop=True)
df_mentioned_user = df_mention['mentioned']
df_User = df_mention['screen_name'].unique()

In [9]:
df_mention.head()

,Unnamed: 0,created_at,text,screen_name,followers,friends,rt,fav,mentioned,quoted,reweet
0,0,Thu Mar 19 21:19:03 +0000 2015,b'@selosul meet up soon for a catch up?',Helenceri,147,295,0,0,Selene O'Sullivan,NaN,NaN
1,2,Thu Sep 11 22:20:13 +0000 2014,"b""@glenniemalcolm that's a crap argument.""",Helenceri,147,295,0,0,Malcolm Glennie,NaN,NaN
2,5,Fri May 09 22:30:54 +0000 2014,"b""@carol_annw :-) c'mon skewen under 7s!""",Helenceri,147,295,1,0,carol webster,NaN,NaN
3,7,Fri May 02 14:37:57 +0000 2014,b'@NPTCouncil @welshgje @Recycle4NPT how do yo...,Helenceri,147,295,0,0,NPT Council,NaN,NaN
4,8,Thu Mar 27 23:29:57 +0000 2014,b'@DrEvanHarris and talking out of her arse. G...,Helenceri,147,295,0,1,Dr Evan Harris,NaN,NaN


In [10]:
tmps = []
for index, row in df_mention.iterrows():
    tmps.append([row.screen_name, row.mentioned, 1])
    

In [11]:
feature_columns = ['twitter_username', 'metioned_user', 'weight']
df_fmetioned_net = pd.DataFrame(data=tmps, columns=feature_columns)

In [12]:
In_degree = df_fmetioned_net.groupby(by=['metioned_user'],as_index=False)['weight'].sum()
Out_degree= df_fmetioned_net.groupby(by=['twitter_username'],as_index=False)['weight'].sum()

In [13]:
In_degree = pd.DataFrame(In_degree).sort_values(by='weight',ascending=False).reset_index(drop=True)
Out_degree = pd.DataFrame(Out_degree).sort_values(by='weight',ascending=False).reset_index(drop=True)

In [14]:
print( 'Most mentioned user is '+str(In_degree['metioned_user'][0])+' with ' + str(In_degree['weight'][0])+' times mentioned by the other users.')
print( 'Most active user is '+str(Out_degree['twitter_username'][0])+' with ' + str(Out_degree['weight'][0])+' times mentioning other users.')

Most mentioned user is Liberal Democrats with 9044 times mentioned by the other users.
Most active user is MikeCrockart with 3400 times mentioning other users.


In [15]:
network1 = df_fmetioned_net.groupby(by=['twitter_username','metioned_user'],as_index=False)['weight'].sum()
network1= pd.DataFrame(network1).sort_values(by='weight',ascending=False).reset_index(drop=True)

In [16]:
print('The 10 most frequent tagged user pairs are ')
network1.iloc[0:19,:]

The 10 most frequent tagged user pairs are 


,twitter_username,metioned_user,weight
0,ScottWood29,CCHQ Press Office,517
1,ChristinaJebb,Liberal Democrats,432
2,JosephBourke,Liberal Democrats,407
3,Ed_Miliband,The Labour Party,398
4,james4suffolk,Liberal Democrats,375
5,RebeccaBlake31,Redditch Standard,311
6,DWard,Liberal Democrats,290
7,HuwBell1,ScotConservatives,289
8,TD_Hancock,jan,287
9,PeterBoneUK,Tom Pursglove MP,279


In [17]:
Out_degree

,twitter_username,weight
0,MikeCrockart,3400
1,MaryMacleodUK,3106
2,emily4MK,2308
3,ScottWood29,2299
4,chrisoxlade,2249
5,Purna_Sen,2242
6,Liam_ascough,2238
7,ClairHawkins,2218
8,ChesterTories,2138
9,LynneBeaumont,2137


In [18]:
len(In_degree)

91330

In [19]:
len(Out_degree)

969

## Mp networks

In [20]:
Mplist = df_fmetioned_net.twitter_username.unique().tolist()

In [21]:
df_MPmetioned_net = df_fmetioned_net[(df_fmetioned_net['metioned_user'].isin(Mplist) == True)]

In [22]:
df_MPmetioned_net

,twitter_username,metioned_user,weight
4494,KevinGillottDCC,olivercoppard,1
12708,DanJarvisMP,olivercoppard,1
12775,DanJarvisMP,KeighleyLabour,1
12776,DanJarvisMP,KeighleyLabour,1
12778,DanJarvisMP,KeighleyLabour,1
12779,DanJarvisMP,KeighleyLabour,1
12901,DanJarvisMP,olivercoppard,1
13022,DanJarvisMP,olivercoppard,1
15294,Ollie_Middleton,olivercoppard,1
15892,Ollie_Middleton,olivercoppard,1


In [23]:
network2 = df_MPmetioned_net.groupby(by=['twitter_username','metioned_user'],as_index=False)['weight'].sum()
network2= pd.DataFrame(network2).sort_values(by='weight',ascending=False).reset_index(drop=True)

In [24]:
print('The 10 most frequent tagged user pairs are ')
network2.iloc[0:19,:]

The 10 most frequent tagged user pairs are 


,twitter_username,metioned_user,weight
0,olivercoppard,olivercoppard,14
1,jackabbott90,olivercoppard,13
2,eddisburylabour,LD4Bassetlaw,9
3,vickyfoxcroft,vickyfoxcroft,8
4,KeighleyLabour,KeighleyLabour,5
5,Steph_Peacock,vickyfoxcroft,4
6,SallyCopley,olivercoppard,4
7,DanJarvisMP,KeighleyLabour,4
8,jon_trickett,KeighleyLabour,4
9,cajardineMP,WAKLibDems,3


In [25]:
G = nx.Graph()
for i in range(len(network1['twitter_username'])):
    G.add_edge(network1['twitter_username'][i],network1['metioned_user'][i],weight=network1['weight'][i])

In [26]:
fi = open('./net.gdf','w')

fi.write('nodedef>name VARCHAR,label VARCHAR\n')
for u in G.nodes():
    fi.write('{},{}\n'.format(u, u))

fi.write('edgedef>node1 VARCHAR,node2 VARCHAR, weight DOUBLE\n')
for i in range(len(network1['twitter_username'])):
    fi.write('{},{},{}\n'.format(network1['twitter_username'][i], network1['metioned_user'][i], network1['weight'][i]*1.0))


In [27]:
fi.close()

In [87]:
nx.write_gexf(G, "network1.gexf")

In [2]:
nx.degree_centrality(G)

NameError: name 'nx' is not defined